In [2]:
import warnings
warnings.simplefilter('ignore')

import pandas as pd
import numpy as np

from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression

<h3>1. Считываем данные по индексу RTSI, Brent, Usd/Rub, S&P 500 за годы 2010-2016</h3>

In [3]:
rtsi_df = pd.read_csv('./data/RTSI_2010-2016.csv')
brent_df = pd.read_csv('./data/Brent_2010-2016.csv')
usdrub_df = pd.read_csv('./data/USD_RUB_2010-2016.csv')
sandp_df = pd.read_csv('./data/SANDP-500_2010-2016.csv')

<h3>2. Избавляемся от ненужных фич, добавляем пропущенные выходные дни и новые фичи</h3>
<h4>Новые фичи:</h4>
<p>Берем биржевые данные по Brent, Usd/Rub и S&P 500. Находим, что произошло с этими инструментами за день до интересующего нас дня. 0 - если цена инструмента за день не изменилась, 1 - если выросла, 2 - если упала. Используем эти данные для предсказания поведения индекса RTSI на следующий день.</p>

In [4]:
def process(df, date_low, date_up):
    df.drop(['<TIME>', '<HIGH>', '<LOW>', '<VOL>'], axis=1, inplace=True)
    
    # 0 - цена не изменилась
    # 1 - цена выросла
    # 2 - цена упала
    df['diff'] = (df['<CLOSE>'] - df['<OPEN>']).astype(float)
    df['result'] = df['diff'].apply(lambda d: 1 if d > 0 else 2 if d < 0 else 0)
    df.drop(['<OPEN>', '<CLOSE>', 'diff'], axis=1, inplace=True)
    
    df.set_index('<DATE>', inplace=True)
    
    idx = pd.date_range(date_low, date_up)
    df.index = pd.DatetimeIndex(df.index)
    df = df.reindex(idx, fill_value=0)
    
    df = df.reset_index()
    df = df.rename(columns = {'index': '<DATE>'})
    
    return df

In [5]:
rtsi_df = process(rtsi_df, '01/01/10', '12/31/2016')

brent_df = process(brent_df, '12/31/09', '12/30/2016')
usdrub_df = process(usdrub_df, '12/31/09', '12/30/2016')
sandp_df = process(sandp_df, '12/31/09', '12/30/2016')

In [6]:
df = rtsi_df
df['brent'] = brent_df['result']
df['usd_rub'] = usdrub_df['result']
df['sandp'] = sandp_df['result']

<h3>3. Форматируем данные</h3>

In [7]:
df['weekday'] = df['<DATE>'].apply(lambda d: d.weekday())
df['month'] = df['<DATE>'].apply(lambda d: d.month)
df.drop('<DATE>', axis=1, inplace=True)

<h3>4. Обучаем модель, используя логистическую регрессию</h3>

In [8]:
y = df['result'].values
df.drop('result', axis=1, inplace=True)

In [9]:
X_train, X_cv, y_train, y_cv = train_test_split(df.values, y, test_size=0.3, random_state=17)

In [10]:
poly = PolynomialFeatures(2)
X_train_poly = poly.fit_transform(X_train)
X_cv_poly = poly.fit_transform(X_cv)

model = LogisticRegression()
model.fit(X_train_poly, y_train)
trainPredict = model.predict(X_train_poly)
testPredict = model.predict(X_cv_poly)

print("LogisticRegression(train) accuracy =", accuracy_score(y_train, trainPredict))
print("LogisticRegression(test) accuracy =", accuracy_score(y_cv, testPredict))

LogisticRegression(train) accuracy = 0.646730016769
LogisticRegression(test) accuracy = 0.66015625


<h3>5. Предсказываем поведение RTSI на завтра</h3>
<p>Вызови из верхнего меню Cell-Run All Above</p>
<p>Задай значения для brent, usd_rub, weekday, month</p>
<p>Запусти нижний блок. 0 - без изменений, 1 - рост, 2 - падение</p>

In [11]:
brent = 1
usd_rub = 2
sandp = 1
weekday = 1
month = 2
X = np.array([brent, usd_rub, sandp, weekday, month])
pred = model.predict(poly.fit_transform(X.reshape(1, -1)))
print(pred[0])

1
